In [6]:
# Implementing the same Surname classifier using CNN

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import re
import os
from torch.utils.data import DataLoader
from argparse import Namespace
import torch
import torch.nn as nn
import torch.optim as optim
# Version descrepancies on this one
from tqdm import tqdm_notebook as notebook

In [8]:
sys.path.append("../cnn_helper/")

In [15]:
%load_ext autoreload
%autoreload 2

from cnn_classifier import SurnameClassifier
from cnn_utils import ModelUtils
from cnn_dataset import SurnameDataset

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
args = Namespace(
surname_csv = "../data/surnames_with_splits.csv",
vectorizer_file = "vectorizer.json",
model_state_file = "model.pth",
save_dir = "../data/model_state",
hidden_dim = 300,
seed = 1337,
num_epochs = 100,
early_stopping_criteria=5,
learning_rate = 0.001,
batch_size=64,
cuda = False,
reload_from_files=False,
expand_filepaths_to_save_dir=True)